# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
F. Walter  ->  F. Walter  |  ['F. Walter']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 51 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2502.14038


Retrieving document from  https://arxiv.org/e-print/2502.14277


/tmp/ipykernel_3201/2822249172.py:52: LatexWarning: 2502.14038 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


extracting tarball to tmp_2502.14277... done.
Retrieving document from  https://arxiv.org/e-print/2502.14539


extracting tarball to tmp_2502.14539...

 done.
Retrieving document from  https://arxiv.org/e-print/2502.14705


extracting tarball to tmp_2502.14705...

 done.
  1: tmp_2502.14705/preamble.tex, 551 lines
  2: tmp_2502.14705/ms.tex, 558 lines


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 2 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2502.14705/ms.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'journals' from 'tmp_2502.14705/journals.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: L

### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.14539-b31b1b.svg)](https://arxiv.org/abs/2502.14539) | **ALMA 360 pc high-frequency observations reveal warm dust in the center of a $z=6.9$ quasar**  |
|| R. A. Meyer, et al. -- incl., <mark>F. Walter</mark> |
|*Appeared on*| *2025-02-21*|
|*Comments*| *5 pages + Appendices. Accepted in A&A Letters*|
|**Abstract**|            The temperature of the cold dust in z>6 galaxies is a potential tracer of Active Galactic Nucleus (AGN) and stellar feedback, and is the dominant source of uncertainty in inferring properties from the far-infrared (FIR) emission of these galaxies. We present the first resolved dust temperature map in a $z>6$ quasar host galaxy. We combine new 360 pc resolution ALMA Band 9 continuum observations with literature 190 pc Band 6 observations to derive the dust temperature and opacity at 0.1<r<0.5 kpc scales in a $z=6.9$ luminous quasar host galaxy (J2348-3054). We find that the dust temperature (and opacity) increases at the center (r<216 pc) of the galaxy up to $T_d=73-88$ K, and potentially up to $T_d<149$ K at r<110 pc. The combination of the resolved and integrated FIR Spectral Energy Distribution (SED) further reveal a dust temperature gradient and a significant contribution of the AGN hot dust torus at $\nu_{\rm{obs}}\gtrsim 700$ GHz. By taking into account the torus contribution and resolved optically-thick emission, we derive a total infrared luminosity ($L_{TIR}=8.78\pm0.10)\times 10^{12}L_\odot$) and corresponding star-formation rate (SFR$=1307\pm15\ M_\odot\ \rm{yr}^{-1}$), that are at least a factor $\sim 3.6$ ($\sim0.56$ dex) lower than previous measurements assuming optically-thin emission. We compare the resolved dust temperature, mass and IR luminosity profiles to simulations where they are only reproduced by models in which the AGN radiation heats the dust in the center of the galaxy. Our observations provide evidence that dust in J2348--3054 cannot be assumed to be uniformly cold and optically thin. Whether J2348-3054 is representative of the larger population of high-redshift quasars and galaxies remains to be determined with dedicated high-resolution and high-frequency ALMA observations.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.14277-b31b1b.svg)](https://arxiv.org/abs/2502.14277) | **Timing and spectral analysis of GK Persei during the 2010 dwarf nova outburst**  |
|| S. Pei, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-02-21*|
|*Comments*| *10 pages, 6 figures*|
|**Abstract**|            GK Persei, an old nova and intermediate polar (IP), exhibited a dwarf nova (DN) outburst in 2010. This outburst was extensively observed by the Neil Gehrels Swift Observatory, beginning 1.95 days after the eruption and continuing until 13.9 days before the maximum of the outburst in the optical. In this paper, we present timing and spectral analyses, comparing the results with those of other outbursts. We confirm the spin modulation in the 2 $-$ 10 keV X-ray range with a period of $P_{\rm WD} = 351.325(9)$ s. Additionally, we detected spin modulation in the 0.3 $-$ 2 keV band during the second half of the observations, a feature not seen in the 2015 and 2018 outbursts. This finding suggests that the soft X-ray emission in GK Per may originate partly near the magnetic poles and partly from a wind or circumstellar material.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.14705-b31b1b.svg)](https://arxiv.org/abs/2502.14705) | **Disturbed cold gas in galaxy and structure formation**  |
|| S. Zou, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-02-21*|
|*Comments*| *21 pages, 7 figures, 1 table in the main text, submitted*|
|**Abstract**|            Cold and cool gas (T $\leq 10^4$ K) in the circumgalactic medium (CGM) and its interaction with galaxies remain poorly understood. Simulations predict that cold gas flows into galaxies through cosmic filaments, determining the disk formation and galaxy evolution. The cold gas accretion modes in the CGM and their dependence on dark matter halo mass and redshift remain puzzling. Resolving the kiloparsec-scale kinematics and dynamics of cold gas interacting with the disk, dust, and metals in different environments is particularly lacking at z > 2. Here we report two disturbed cold gas structures traced by ultra-strong MgII absorbers (rest-frame equivalent width Wr > 2 Å) exhibiting high kinematic velocities (> 500 km/s) and their environments at z ~ 4.9 and z ~ 2.6. Observations were conducted with VLT/MUSE, JWST/NIRCam, and ALMA to detect Lya and nebular emission lines, as well as dust continuum emission in the vicinity of these two absorbing gas structures. We identify two Lya emitters associated with a strong MgII absorber pair separated by ~1000 km/s at z~ 4.87. The pair exhibits relative differences in metallicity, dust content, and ionization states, suggesting internal metal and dust exchange within the ultra-large cold gas structure. For the strong MgII absorber at z = 2.5652$, we detect a dusty star-forming galaxy at a projected distance of $D = 38$ kpc. This galaxy exhibits prominent HeI, [SIII], and Paschen$\gamma$ lines, along with significant dust continuum. It has a star formation rate of ~ 121 +/- 33 $M_{\odot}$/yr and likely harbors a rotating disk. These findings tentatively suggest that cold gas at high redshifts plays a critical role in driving disk formation and actively participates in the transfer of metals and dust within the overdense regions of the CGM.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2502.14038-b31b1b.svg)](https://arxiv.org/abs/2502.14038) | **Three Years of High-Contrast Imaging of the PDS 70 b and c Exoplanets at Hα with MagAO-X: Evidence of Strong Protoplanet Hα Variability and Circumplanetary Dust**  |
|| L. M. Close, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-02-21*|
|*Comments*| *50 pages, 20 figures, published in January 2025 Astronomical Journal. NOTE Figure D1 has correct planetary mass accretion scale compared to published AJ version*|
|**Abstract**|            We present 3 years of high-contrast imaging of the PDS 70 b and c accreting protoplanets with the new extreme AO system MagAO-X as part of the MaxProtoPlanetS survey of H$\alpha$ protoplanets. In 2023 and 2024 our sharp (25-27 mas FWHM); well AO corrected (20-26% Strehl), deep (2-3.6hr) images detect compact (r~30 mas; r~3 au) circumplanetary disks (CPDs) surrounding both protoplanets. Starlight scattering off the dusty outer edges of these CPDs is the likely source of the bright compact continuum light detected within ~30 mas of both planets in our simultaneously obtained continuum 668 nm filter images. After subtraction of contaminating continuum and PSF residuals with pyKLIP ADI and SDI we obtained high-contrast ASDI H$\alpha$ images of both planets in 2022, 2023 and 2024. We find the H$\alpha$ line flux of planet b fell by (8.1$\pm$1.6)x10$^{-16}$ ergs/s/cm$^2$ a factor of 4.6 drop in flux from 2022 to 2023. In March 2024, planet b continued to be faint with just a slight 1.6x rise to an H$\alpha$ line flux of (3.64$\pm$0.87)x10$^{-16}$ ergs/s/cm$^2$. For c we measure a significant increase of (2.74$\pm$0.51)x10$^{-16}$ ergs/s/cm$^2$ from 2023 to 2024 which is a factor of 2.3x increase. So both protoplanets have recently experienced significant H$\alpha$ variability with ~1 yr sampling. In 2024, planet c is brighter than b: as c is brightening and b generally fading. We also tentatively detect one new point source "CC3" inside the inner disk (~49 mas; at PA~295 deg; 2024) with orbital motion roughly consistent with a ~5.6 au orbit.         |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

339  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
